In [2]:
from __future__ import annotations
import datetime
from time import time
from tkinter import E

def connect_binance():
    """Connect to binance and return the client"""
    import binance
    from binance.client import Client
    client = Client("#", "#")
    return client

def create_a_csv():
    """Create a csv file"""
    import time
    import pandas as pd
    df = pd.DataFrame(columns=['Current', 'Balance', 'Value', 'Benefits', 'Date'])
    df.loc[len(df)] = [3, 100, 610004, 0,  '25/02/2022']
    #Delete the index column
    df.to_csv('current.csv')

def get_klines(client, symbol, interval, start_str, end_str):
    """Get klines from binance and return the data"""
    klines = client.get_historical_klines(symbol, interval, start_str, end_str)
    return klines

def klines_to_df(klines):
    """Convert klines to a plot candle chart"""
    import pandas as pd
    from datetime import datetime
    
    #Transform klines to a dataframe=
    transformed_data = []
    for i in range(len(klines)):
        date = datetime.fromtimestamp(int(klines[i][0])/1000).strftime('%Y-%m-%d %H:%M:%S')
        transformed_data.append([date, klines[i][1], klines[i][2], klines[i][3], klines[i][4], klines[i][2], klines[i][2], klines[i][2], klines[i][2], klines[i][2], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    #Trasform transformed data to a dataframe
    df = pd.DataFrame(transformed_data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b', 
    'chikou_span', 'MA3','MA10', 'MA25', 'MA50', 'MA100', 'MA200', 'EMA5', 'EMA10','EMA50', 'EMA100', 
    'EMA200', 'Val-1', 'Val-2', 'Val-3', 'Val-4', 'Val-5', 'final'])
    #Set the index to the date
    df.index = df['Date']
    #Set the column MA to float
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['tenkan_sen'] = df['tenkan_sen'].astype(float)
    df['kijun_sen'] = df['kijun_sen'].astype(float)
    df['senkou_span_a'] = df['senkou_span_a'].astype(float)
    df['senkou_span_b'] = df['senkou_span_b'].astype(float)
    df['chikou_span'] = df['chikou_span'].astype(float)
    df['MA3'] = df['MA3'].astype(float)
    df['MA10'] = df['MA10'].astype(float)
    df['MA25'] = df['MA25'].astype(float)
    df['MA50'] = df['MA50'].astype(float)
    df['MA100'] = df['MA100'].astype(float)
    df['MA200'] = df['MA200'].astype(float)
    df['EMA5'] = df['EMA5'].astype(float)
    df['EMA10'] = df['EMA10'].astype(float)
    df['EMA50'] = df['EMA50'].astype(float)
    df['EMA100'] = df['EMA100'].astype(float)
    df['EMA200'] = df['EMA200'].astype(float)
    df['Val-1'] = df['Val-1'].astype(float)
    df['Val-2'] = df['Val-2'].astype(float)
    df['Val-3'] = df['Val-3'].astype(float)
    df['Val-4'] = df['Val-4'].astype(float)
    df['Val-5'] = df['Val-5'].astype(float)
    df['final'] = df['final'].astype(float)
    return df


      
def turn_on():
    import warnings
    warnings.filterwarnings("ignore")
    import pandas as pd
    import numpy as np
    np.seterr(divide='ignore', invalid='ignore')
    client = connect_binance()
    now = datetime.datetime.now()
    bitcoin_klines = get_klines(client, "BTCUSDT", client.KLINE_INTERVAL_30MINUTE, "1 Jan, 2021", "20 Dec, 2022")
    fake = klines_to_df(bitcoin_klines)
    #fake = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b', 'chikou_span', 'MA3','MA10', 'MA25', 'MA50', 'MA100', 'MA200', 'EMA5', 'EMA10','EMA50', 'EMA100', 'EMA200', 'final'])
    tenkan_sen_high = fake['High'].rolling( window=9 ).max()
    tenkan_sen_low = fake['Low'].rolling( window=9 ).min()
    fake['tenkan_sen'] = (tenkan_sen_high + tenkan_sen_low) /2
    kijun_sen_high = fake['High'].rolling( window=26 ).max()
    kijun_sen_low = fake['Low'].rolling( window=26 ).min()
    fake['kijun_sen'] = (kijun_sen_high + kijun_sen_low) /2
    fake['senkou_span_a'] = ((fake['tenkan_sen'] + fake['kijun_sen']) / 2).shift(26)
    period52_high = fake['High'].rolling(window=52).max()
    period52_low = fake['Low'].rolling(window=52).min()
    fake['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    fake['chikou_span'] = fake['Close'].shift(-26)
    print('ichimoku done')
    fake['MA3'] = fake['Close'].rolling(window=3).mean()
    fake['MA10'] = fake['Close'].rolling(window=10).mean()
    fake['MA25'] = fake['Close'].rolling(window=25).mean()
    fake['MA50'] = fake['Close'].rolling(window=50).mean()
    fake['MA100'] = fake['Close'].rolling(window=100).mean()
    fake['MA200'] = fake['Close'].rolling(window=200).mean()
    print("MA done")
    fake['EMA5'] = fake['Close'].ewm(span=5, adjust=False).mean()
    fake['EMA10'] = fake['Close'].ewm(span=10, adjust=False).mean()
    fake['EMA50'] = fake['Close'].ewm(span=50, adjust=False).mean()
    fake['EMA100'] = fake['Close'].ewm(span=100, adjust=False).mean()
    fake['EMA200'] = fake['Close'].ewm(span=200, adjust=False).mean()
    print("EMA done")
    fake['Val-1'] = fake['Close'].shift(1)
    fake['Val-2'] = fake['Close'].shift(2)
    fake['Val-3'] = fake['Close'].shift(3)
    fake['Val-4'] = fake['Close'].shift(4)
    fake['Val-5'] = fake['Close'].shift(5)
    #final is equal to the MA of the next 5 rows
    fake['final'] = fake['MA25'].shift(-5)
    print("final done")
    #Delete the chikou_span column
    del fake['chikou_span']
    #Delete the first and second column
    fake = fake.iloc[:,1:]
    #Delete the last 26 rows
    fake = fake.iloc[:-26]
    #Delete the first 200 rows
    fake = fake.iloc[200:]

    fake.to_csv('fake.csv')
    print("done")

turn_on()

ichimoku done
MA done
EMA done
final done
done
